## Importing the Data

In [1]:
# Constants
DATASET_DIR = './data/'
GLOVE_DIR = './glove.6B/'
SAVE_DIR = './'

import os
import pandas as pd

X = pd.read_csv(os.path.join(DATASET_DIR, 'training_set_rel3.tsv'), sep='\t', encoding='ISO-8859-1')
y = X['domain1_score']
X = X.dropna(axis=1)
X = X.drop(columns=['rater1_domain1', 'rater2_domain1'])

In [2]:
X.head()

,essay_id,essay_set,essay,domain1_score
0,1,1,"Dear local newspaper, I think effects computer...",8
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",10
4,5,1,"Dear @LOCATION1, I know having computers has a...",8


Minimum and Maximum Scores for each essay set.

In [3]:
minimum_scores = [-1, 2, 1, 0, 0, 0, 0, 0, 0]
maximum_scores = [-1, 12, 6, 3, 3, 4, 4, 30, 60]

## Preprocessing the Data

We will preprocess all essays and convert them to feature vectors so that they can be fed into the RNN.

These are all helper functions used to clean the essays.

In [4]:
import numpy as np
import nltk
import re
from nltk.corpus import stopwords
from gensim.models import Word2Vec

def essay_to_wordlist(essay_v, remove_stopwords):
    """Remove the tagged labels and word tokenize the sentence."""
    essay_v = re.sub("[^a-zA-Z]", " ", essay_v)
    words = essay_v.lower().split()
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    return (words)

def essay_to_sentences(essay_v, remove_stopwords):
    """Sentence tokenize the essay and call essay_to_wordlist() for word tokenization."""
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    raw_sentences = tokenizer.tokenize(essay_v.strip())
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append(essay_to_wordlist(raw_sentence, remove_stopwords))
    return sentences

def makeFeatureVec(words, model, num_features):
    """Make Feature Vector from the words list of an Essay."""
    featureVec = np.zeros((num_features,),dtype="float32")
    num_words = 0.
    index2word_set = set(model.wv.index_to_key)
    for word in words:
        if word in index2word_set:
            num_words += 1
            featureVec = np.add(featureVec,model.wv[word])        
    featureVec = np.divide(featureVec,num_words)
    return featureVec

def getAvgFeatureVecs(essays, model, num_features):
    """Main function to generate the word vectors for word2vec model."""
    counter = 0
    essayFeatureVecs = np.zeros((len(essays),num_features),dtype="float32")
    for essay in essays:
        essayFeatureVecs[counter] = makeFeatureVec(essay, model, num_features)
        counter = counter + 1
    return essayFeatureVecs

## Defining the model 

Here we define a 2-Layer LSTM Model. 

Note that instead of using sigmoid activation in the output layer we will use
Relu since we are not normalising training labels.

In [5]:
from keras.layers import Embedding, LSTM, Dense, Dropout, Conv1D, Lambda, Flatten, MaxPooling1D
from keras.models import Sequential
import keras.regularizers
import keras.backend as K

def get_model():
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=5, padding='same'))
    model.add(MaxPooling1D(2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='relu', activity_regularizer=keras.regularizers.l2(0.0)))
    model.compile(loss='mean_squared_error', optimizer='adam')

    return model

2024-11-22 20:18:29.335829: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-22 20:18:29.345467: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-22 20:18:29.413902: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-22 20:18:29.507493: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-22 20:18:29.613815: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registe

## Training Phase

Now we train the model on the dataset.

We will use 5-Fold Cross Validation and measure the Quadratic Weighted Kappa for each fold.
We will then calculate Average Kappa for all the folds.

In [ ]:
from sklearn.model_selection import KFold
# from sklearn.linear_model import LinearRegression
from sklearn.metrics import cohen_kappa_score
# nltk.download('stopwords')
cv = KFold(5, shuffle=True, random_state=690420)
results = []
y_pred_list = []

count = 1
for traincv, testcv in cv.split(X, y):
    print("\n--------Fold {}--------\n".format(count))
    X_test, X_train, y_test, y_train = X.iloc[testcv], X.iloc[traincv], y.iloc[testcv], y.iloc[traincv]
    
    train_essays = X_train['essay']
    test_essays = X_test['essay']
    
    sentences = []
    
    for essay in train_essays:
            # Obtaining all sentences from the training essays.
            sentences += essay_to_sentences(essay, remove_stopwords = True)
            
    # Initializing variables for word2vec model.
    num_features = 300 
    min_word_count = 40
    num_workers = 4
    context = 10
    downsampling = 1e-3

    print("Training Word2Vec Model...")
    model = Word2Vec(sentences, workers=num_workers, vector_size=num_features, min_count = min_word_count, window = context, sample = downsampling)

    model.init_sims(replace=True)
    model.wv.save_word2vec_format('./mysite/grader/deep_learning_files/word2vecmodel.bin', binary=True)

    clean_train_essays = []
    
    # Generate training and testing data word vectors.
    for essay_v in train_essays:
        clean_train_essays.append(essay_to_wordlist(essay_v, remove_stopwords=True))
    trainDataVecs = getAvgFeatureVecs(clean_train_essays, model, num_features)
    
    clean_test_essays = []
    for essay_v in test_essays:
        clean_test_essays.append(essay_to_wordlist( essay_v, remove_stopwords=True ))
    testDataVecs = getAvgFeatureVecs( clean_test_essays, model, num_features )
    
    trainDataVecs = np.array(trainDataVecs)
    testDataVecs = np.array(testDataVecs)
    # Reshaping train and test vectors to 3 dimensions. (1 represnts one timestep)
    trainDataVecs = np.reshape(trainDataVecs, (trainDataVecs.shape[0], trainDataVecs.shape[1], 1))
    testDataVecs = np.reshape(testDataVecs, (testDataVecs.shape[0], testDataVecs.shape[1], 1 ))
    
    lstm_model = get_model()
    lstm_model.fit(trainDataVecs, y_train, batch_size=64, epochs=50)
    #lstm_model.load_weights('./model_weights/final_lstm.h5')
    y_pred = lstm_model.predict(testDataVecs)
    
    # Round y_pred to the nearest integer.
    y_pred = np.around(y_pred)
    
    # Evaluate the model on the evaluation metric. "Quadratic mean averaged Kappa"
    result = cohen_kappa_score(y1=y_test,y2=y_pred,weights='quadratic')

    print("Kappa Score: {}".format(result))
    results.append(result)

    count += 1
        


--------Fold 1--------

Training Word2Vec Model...


/tmp/ipykernel_3865/1640451972.py:33: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Epoch 1/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 6s 23ms/step - loss: 79.0862
Epoch 2/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 34.4621
Epoch 3/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 19.9607
Epoch 4/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 15.4904
Epoch 5/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 12.2867
Epoch 6/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 11.3507
Epoch 7/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 11.4878
Epoch 8/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 10.4821
Epoch 9/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 8.8336
Epoch 10/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 9.8726
Epoch 11/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - loss: 9.9039
Epoch 12/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - loss: 9.2711
Epoch 13/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 8.8712
Epoch 14/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 8.5015
Epoch 15/50
163/163 ━━━━━━━━━━━━━━━

/tmp/ipykernel_3865/1640451972.py:33: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Epoch 1/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 6s 23ms/step - loss: 72.7501
Epoch 2/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - loss: 32.4205
Epoch 3/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 15.5383
Epoch 4/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - loss: 12.1105
Epoch 5/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 11.2758
Epoch 6/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 10.8185
Epoch 7/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 9.5766
Epoch 8/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 10.8948
Epoch 9/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 10.1190
Epoch 10/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 8.9580
Epoch 11/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 8.3621
Epoch 12/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 8.6562
Epoch 13/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 9.2779
Epoch 14/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 8.8399
Epoch 15/50
163/163 ━━━━━━━━━━━━━━━

...

/tmp/ipykernel_3865/1640451972.py:33: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Epoch 1/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 6s 23ms/step - loss: 78.3169
Epoch 2/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - loss: 33.5429
Epoch 3/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 21.1786
Epoch 4/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 5s 23ms/step - loss: 14.7721
Epoch 5/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 12.8182
Epoch 6/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - loss: 11.8042
Epoch 7/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 10.9306
Epoch 8/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 9.6804
Epoch 9/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 9.9589
Epoch 10/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 9.6694
Epoch 11/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 10.0545
Epoch 12/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 9.0323
Epoch 13/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 9.2521
Epoch 14/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 9.4477
Epoch 15/50
163/163 ━━━━━━━━━━━━━━━

/tmp/ipykernel_3865/1640451972.py:33: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Epoch 1/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 6s 23ms/step - loss: 79.5877
Epoch 2/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 34.5819
Epoch 3/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 18.6691
Epoch 4/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 13.5518
Epoch 5/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 12.3584
Epoch 6/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - loss: 12.2748
Epoch 7/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 11.6886
Epoch 8/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 10.0968
Epoch 9/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 10.4391
Epoch 10/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 9.1673
Epoch 11/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 9.6655
Epoch 12/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 9.0028
Epoch 13/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 9.3105
Epoch 14/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 8.3159
Epoch 15/50
163/163 ━━━━━━━━━━━━━━

Kappa Score: 0.9677733859248027


The Avg. Kappa Score is 0.961 which is the highest we have ever seen on this dataset.

In [7]:
print("Average Kappa score after a 5-fold cross validation: ",np.around(np.array(results).mean(),decimals=4))

Average Kappa score after a 5-fold cross validation:  0.9644
